In [19]:
import sys
from pathlib import Path

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        newp = Path(this_path).parent
        # check if src folder:
        src = newp.joinpath("src")
        if src.exists():
            newp = str(src)
        else:
            newp = str(newp)
    else:
        newp = str(Path(this_path)) 

    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))

def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath     

    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

#DIR_DATA, DIR_IMG = get_project_dirs()

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-v0_8-muted')
from pprint import pprint as pp

# Filtered dir() for method discovery:
def fdir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import HTML, Markdown #, IFrame

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

---

# MCCE - MS Sampling
---

In [237]:
import base
import mcce_io as io
import ms_sampling

In [143]:
DATA = Path.cwd().parent.joinpath("tests/data")
DATA

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data')

In [144]:
!ls {DATA}

head3.lst  ms_out  run.prm.record  step2_out.pdb


In [ ]:
# filepaths of inputs:
h3_path = DATA.joinpath("head3.lst")
mcce_output_path = h3_path.parent

step2_path = mcce_output_path.joinpath("step2_out.pdb")
msout_path = mcce_output_path.joinpath("ms_out")
msout_path, msout_path.is_dir()


In [246]:
# filepaths of outputs:

ph = 5.0
eh = 0.0
fname = io.get_msout_filename(mcce_output_path, ph, eh)
fname

pdb_out_folder = fname.parent.joinpath(fname.stem)
pdb_out_folder

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms.txt')

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms')

In [ ]:
conformers = io.read_conformers(h3_path)

ms = base.MS(mcce_output_path, 5., 0.)

ms_orig_lst = [Micro_tup(ms.E, ms.count, ms.state) for ms in list((ms.microstates.values()))]
print(f"# microstates: {len(ms_orig_lst):,}")

ms_sorted_lst = ms_sampling.sort_microstate_list(ms_orig_lst, by="energy")

In [239]:
N_samples = 2
ms_cumsum, selection_range = get_regular_samples(N_samples, ms_sorted_lst)

for count in selection_range:
    ms_index = np.where((ms_cumsum - count) > 0)[0][0]
    selected_ms = ms_sampling.ms_sorted_lst[ms_index]
    selected_confs = ms_sampling.get_selected_confs(ms, conformers, selected_ms)
    #continue
    
    # write out the pdb
    #io.MS_to_PDB(selected_confs, ms_index, step2_path, output_folder)

print("PDB creation from sampled ms over!")

PDB creation from sampled ms over!
